## HOPR Token holders 

In [1]:
import time
import pandas as pd
import os 
from requests import get, post
import requests
from load_dotenv import load_dotenv 


In [5]:
# Load variables 
load_dotenv('secrets.env')

# dune
api_key_dune = os.environ['API_KEY_DUNE']
HEADER = {"x-dune-api-key" : api_key_dune}
BASE_URL = "https://api.dune.com/api/v1/"

# hopr node
api_host_hopr_node = os.environ['API_HOST_HOPR_NODE']
api_key_hopr_node = os.environ['API_KEY_HOPR_NODE']

if 'API_KEY_DUNE' in os.environ and 'API_HOST_HOPR_NODE' in os.environ and 'API_KEY_HOPR_NODE' in os.environ:
    print(True)
else:
    print(False)

True


### Define functions to extract data from Dune API

In [55]:
def make_api_url(module, action, ID):
    """
    Generate a URL to call the API.
    """

    url = BASE_URL + module + "/" + ID + "/" + action

    return url

In [56]:
def execute_query(query_id, engine="medium"):
    """
    Takes in the query ID and engine size.
    Specifying the engine size will change how quickly your query runs. 
    The default is "medium" which spends 10 credits, while "large" spends 20 credits.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """

    url = make_api_url("query", "execute", query_id)
    params = {
        "performance": engine,
    }
    response = post(url, headers=HEADER, params=params)
    execution_id = response.json()['execution_id']

    return execution_id


def get_query_status(execution_id):
    """
    Takes in an execution ID.
    Fetches the status of query execution using the API
    Returns the status response object
    """

    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)

    return response


def get_query_results(execution_id):
    """
    Takes in an execution ID.
    Fetches the results returned from the query using the API
    Returns the results response object
    """

    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)

    return response


def cancel_query_execution(execution_id):
    """
    Takes in an execution ID.
    Cancels the ongoing execution of the query.
    Returns the response object.
    """

    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)

    return response


In [57]:
# query data 
execution_id = execute_query("3244002","medium")
print(execution_id)

01HGJXJQEW0FMDQPRZ0FPT6R84


In [58]:
response_status = get_query_status(execution_id)
# print(response_status.json())

In [59]:
def wait_for_query_completion(execution_id, interval=5, timeout=600):
    """
    Waits for a query to complete and prints the final state and returns the results.
    :param: execution_id: str, the ID of the query execution.
    :param: interval (int): the interval (in seconds) between status checks.
    :returns: final response status.
    """
    start_time = time.time()

    # Initial request to get the query status
    response_status = get_query_status(execution_id)
    state = response_status.json()['state']

    # Continue checking the state until it becomes 'QUERY_STATE_COMPLETED'
    while state == 'QUERY_STATE_EXECUTING' and (time.time() - start_time) < timeout:
        time.sleep(interval)
        response_status = get_query_status(execution_id)
        state = response_status.json()['state']

    print("Final state:", state)
    
    return response_status.json()


In [60]:
wait_for_query_completion(execution_id, interval=5, timeout=600)

Final state: QUERY_STATE_COMPLETED


{'execution_id': '01HGJXJQEW0FMDQPRZ0FPT6R84',
 'query_id': 3244002,
 'state': 'QUERY_STATE_COMPLETED',
 'submitted_at': '2023-12-01T14:47:31.549445Z',
 'expires_at': '2024-02-29T14:52:28.683829Z',
 'execution_started_at': '2023-12-01T14:47:31.558741818Z',
 'execution_ended_at': '2023-12-01T14:52:28.68382804Z',
 'result_metadata': {'column_names': ['wallet',
   'total_hopr_balance',
   'is_hopr_safe',
   'node_address'],
  'result_set_bytes': 289,
  'total_row_count': 3,
  'datapoint_count': 12,
  'pending_time_millis': 9,
  'execution_time_millis': 297125}}

In [61]:
response_result = get_query_results(execution_id)
print(response_result)

<Response [200]>


In [62]:
data = pd.DataFrame(response_result.json()['result']['rows'])
data.head()

,is_hopr_safe,node_address,total_hopr_balance,wallet
0,Yes,0x31faaa8ce2fe3a22eab7c05167312f853adc0042,158650.000000,0xcd1a97453e3525019cdfa66fb669576fafb2c527
1,No,None,50822.634100,0x217a6d29abbaceafb36207b4cb25acc148e1fc65
2,No,None,54380.632024,0xd9a00176cf49dfb9ca3ef61805a2850f45cb1d05


### Funds deposited in payment channels 

In [3]:
def get_unique_nodeAddress_peerId_aggbalance_links(api_host_hopr_node, api_key_hopr_node):
    """
    Returns a dict containing all unique source_peerId-source_address links.
    """
    channel_url = "http://{}:3001/api/v3/channels/?includingClosed=false&fullTopology=true".format(api_host_hopr_node)
    headers = {'X-Auth-Token': api_key_hopr_node}
    response = requests.request("GET", channel_url, headers=headers)

    if response.status_code != 200:
        print("Could not fetch channel information. Status code: {}".format(response.status_code))
        return {}

    response = response.json()

    if 'all' not in response:
            print("Response does not contain `all`")
            return {}

    peerid_address_aggbalance_links = {}
    for item in response["all"]:
        if "sourcePeerId" not in item or "sourceAddress" not in item:
            print("Response does not contain `source_peerid` or `source_address`")
            continue

        if "status" not in item:
            print("Response does not contain `status`")
            continue

        source_peer_id = item["sourcePeerId"]
        source_address = item["sourceAddress"]
        balance = int(item["balance"]) / 1e18

        if item["status"] != "Open":
            # Other Statuses: "Waiting for commitment", "Closed", "Pending to close"
            continue

        if source_peer_id not in peerid_address_aggbalance_links:
            peerid_address_aggbalance_links[source_peer_id] = {
                "source_node_address": source_address,
                "channels_balance": balance,
            }

        else:
            peerid_address_aggbalance_links[source_peer_id][
                "channels_balance"
            ] += balance

    return peerid_address_aggbalance_links

In [6]:
topology_data = get_unique_nodeAddress_peerId_aggbalance_links(api_host_hopr_node, api_key_hopr_node)

print(len(topology_data))

for key, value in list(topology_data.items())[:5]:
    print(f"{key}: {value}")

398
12D3KooWRbkemN63B4kirDtcEUH6pegkNHV5TxdJRX4LYMgyLysF: {'source_node_address': '0xf7f1f61f8c39b3034e294887e76c6b1ccee2f9e1', 'channels_balance': 1.949999999999996e-10}
12D3KooWAAZ3WnaWMuSQnQSFYnwx7g6W725DYmDY1QASZK1nni5B: {'source_node_address': '0xfa73af5609d354acd0f6e51f77c04d228c370813', 'channels_balance': 460.0}
12D3KooWC7Tub3aRmxUM7XjiozW5paXcySN9gWDDvJwNh98DTdUD: {'source_node_address': '0x07a52c8338edef4b6d3a180c1b75815469ff3a46', 'channels_balance': 1081.0}
12D3KooWNJyWUuu9M9RNU8zTMyqCA8K1GqJwv8tjHsc2oYsJJajD: {'source_node_address': '0x9925b9846e6e289e1c7f8a0554bf6900757491ec', 'channels_balance': 226.2}
12D3KooWKn3bLWMymSMznYt3YRrK8cwmYytEzwM1HoDgqkLcmeSu: {'source_node_address': '0x8de95cdc3291152381f24226422ed9c08beec146', 'channels_balance': 3546.8}
